In [20]:
import math
import datetime
###### PI value
DPI = 3.141592653589793

In [14]:
def parse_date_to_YMD_flexible(date_string: str) -> tuple[int, int, float]:
    """
    Parses a date string in "YYYY-MM-DD HH:MM" or "YYYY-MM-DD HH:MM:SS.SS" format 
    and returns (Y, M, D) where D includes the time as a decimal fraction of the day.
    
    Args:
        date_string (str): The date and time string.
        
    Returns:
        tuple[int, int, float]: (Year, Month, Day with decimal time).
    """
    
    # 1. Split the date and time parts
    try:
        date_part, time_part = date_string.split(' ')
    except ValueError:
        raise ValueError(f"Invalid format. Must contain a space between date and time: {date_string}")
    
    # 2. Extract Y, M, and the base day
    try:
        Y, M, day = map(int, date_part.split('-'))
    except ValueError:
        raise ValueError(f"Invalid date format: {date_part}. Expected YYYY-MM-DD.")
    
    # 3. Parse the time part and calculate total seconds
    
    # Split HH:MM[:SS.SS]
    time_components = time_part.split(':')
    H = int(time_components[0])
    Min = int(time_components[1]) # Minutes must be an integer part
    
    total_seconds = float(H * 3600) + float(Min * 60)
    
    if len(time_components) == 3:
        # Format is HH:MM:SS.SS
        # The last component can be a float (e.g., 55.45)
        Sec_decimal = float(time_components[2])
        total_seconds += Sec_decimal
        
    elif len(time_components) != 2:
        # Neither HH:MM nor HH:MM:SS.SS
        raise ValueError(f"Invalid time format: {time_part}. Expected HH:MM or HH:MM:SS.SS.")

    # 4. Calculate the fraction of the day
    seconds_in_day = 86400.0 # 24 * 60 * 60
    decimal_fraction_of_day = total_seconds / seconds_in_day
    
    # 5. Combine base day and decimal fraction
    D = float(day) + decimal_fraction_of_day
    
    return (Y, M, D)

def calculate_julian_day(Y: int, M: int, D: float) -> float:
    """
    Calculates the Julian Day (JD) for a given calendar date, 
    valid for positive and negative years (but not negative JD).
    
    Args:
        Y (int): Year.
        M (int): Month number (1=Jan, 12=Dec).
        D (float): Day of the month (with decimals for time).
        
    Returns:
        float: The calculated Julian Day.
    """
    
    # 1. Adjust Y and M for January and February
    # (M=1 or M=2 become month 13 or 14 of the preceding year)
    if M <= 2:
        Y = Y - 1
        M = M + 12
    
    # 2. Determine the value of B based on the calendar
    
    # The transition from Julian to Gregorian occurred after 4 Oct 1582.
    # The day after 4 Oct 1582 was 15 Oct 1582 (10 days were skipped).
    # We must check the date against the transition point: Y=1582, M=10, D>=15.
    
    # Check for the Gregorian calendar (after 14 Oct 1582)
    # The condition is: Y > 1582, OR (Y == 1582 AND M > 10), OR (Y == 1582 AND M == 10 AND D >= 15).

    # A more direct date comparison:
    # Gregorian starts on 15 October 1582.
    is_gregorian = False
    if Y > 1582:
        is_gregorian = True
    elif Y == 1582:
        if M > 10:
            is_gregorian = True
        elif M == 10 and D >= 15:
            is_gregorian = True
            
    # Calculate B
    if is_gregorian:
        # Gregorian Calendar calculation (after 14 Oct 1582)
        A = math.floor(Y / 100)
        B = 2 - A + math.floor(A / 4)
    else:
        # Julian Calendar calculation (prior to 15 Oct 1582)
        B = 0
        
    # 3. Calculate the Julian Day (JD)
    
    # Formula (7.1):
    # JD = INT(365.25(Y + 4716)) + INT(30.6001(M + 1)) + D + B - 1524.5
    
    term1 = math.floor(365.25 * (Y + 4716))
    term2 = math.floor(30.6001 * (M + 1))
    
    JD = term1 + term2 + D + B - 1524.5
    
    return JD


def calculate_mean_sidereal_time(JD: float) -> float:
    """
    Calculates the Mean Sidereal Time at Greenwich (theta_0) in degrees 
    using the full expression of equation (11.4).

    Args:
        JD (float): The Julian Day (JD) for the instant of Universal Time (UT). 
                    This must be the instantaneous JD, not just the JD for 0h UT.

    Returns:
        float: The Mean Sidereal Time at Greenwich (theta_0) in decimal degrees, 
               normalized to the range [0.0, 360.0).
    """
    
    # JD_2000_EPOCH = 2451545.0 (Julian Day for 12h UT on Jan 1, 2000)
    JD_DIFF = JD - 2451545.0
    
    # 1. Calculate T (Time in Julian centuries since J2000.0)
    # T = (JD - 2451545.0) / 36525.0  (based on equation 11.1)
    T = JD_DIFF / 36525.0
    
    # 2. Calculate theta_0 (Mean Sidereal Time at Greenwich) using equation (11.4)
    # theta_0 = 280.46061837 + 360.98564736629(JD - 2451545.0) 
    #         + 0.000387933 * T^2 - T^3 / 38710000
    
    # Term 1: Constant
    term1 = 280.46061837
    
    # Term 2: Linear change over time
    term2 = 360.98564736629 * JD_DIFF
    
    # Term 3: Quadratic correction (due to T^2)
    term3 = 0.000387933 * (T ** 2)
    
    # Term 4: Cubic correction (due to T^3)
    term4 = (T ** 3) / 38710000.0
    
    theta_0 = term1 + term2 + term3 - term4
    
    # 3. Normalize the result to the range [0.0, 360.0)
    # Use the modulo operator to reduce the degrees
    theta_0 = theta_0 % 360.0
    
    # Ensure the result is positive, as sometimes the modulo can return a small negative value
    if theta_0 < 0:
        theta_0 += 360.0
        
    return theta_0

def degrees_to_hms(degrees: float) -> tuple[int, int, float]:
    """
    Converts a decimal degree value (like the result of equation 11.4) 
    into Hours, Minutes, and Seconds (H:M:S).
    
    Args:
        degrees (float): The angle in decimal degrees.
        
    Returns:
        tuple[int, int, float]: (Hours, Minutes, Seconds).
    """
             
    # 2. Convert degrees to decimal hours by dividing by 15
    decimal_hours = degrees / 15.0
    
    # 3. Extract Hours
    Hours = math.floor(decimal_hours)
    
    # 4. Extract Minutes
    # Get the fractional part of hours and multiply by 60
    decimal_minutes = (decimal_hours - Hours) * 60.0
    Minutes = math.floor(decimal_minutes)
    
    # 5. Extract Seconds
    # Get the fractional part of minutes and multiply by 60
    Seconds = (decimal_minutes - Minutes) * 60.0
    
    # Ensure seconds is float for decimal precision
    return (int(Hours), int(Minutes), Seconds)


def parse_ra_string(ra_string):
    """
    Parse right ascension string into component values.
    
    Takes a right ascension string in format "HH MM SS.SS" and extracts
    the hours, minutes, and seconds as separate float values.
    
    Args:
        ra_string (str): Right ascension in format "HH MM SS.SS"
                        (e.g., "14 23 45.67")
    
    Returns:
        tuple: A tuple of three floats (hours, minutes, seconds)
               e.g., (14.0, 23.0, 45.67)
    
    Example:
        >>> parse_ra_string("14 23 45.67")
        (14.0, 23.0, 45.67)
    """

    try:
        parts = ra_string.strip().split()
        if len(parts) != 3:
            raise ValueError("RA string must be in the format 'HH MM SS.SS'")
        
        ra_hours = float(parts[0])
        ra_minutes = float(parts[1])
        ra_seconds = float(parts[2])
        
        return (ra_hours, ra_minutes, ra_seconds)
    
    except ValueError as e:
        print(f"Error parsing RA string: {e}")
        return None
    

def convert_ra_components(hours, minutes, seconds):
    """
    Convert right ascension components to decimal, degrees, and radians.
    
    Takes individual hours, minutes, and seconds components of right ascension
    and converts them to decimal hours, decimal degrees, and radians.
    
    Args:
        hours (float): Hours component of right ascension
        minutes (float): Minutes component of right ascension  
        seconds (float): Seconds component of right ascension
    
    Returns:
        tuple: A tuple of three floats:
               - decimal_hours (float): RA in decimal hours
               - degrees (float): RA in decimal degrees
               - radians (float): RA in radians
    
    Example:
        >>> convert_ra_components(14.0, 23.0, 45.67)
        14.396019444444445, 215.9402916666667, 3.768...
    
    Note:
        Right ascension ranges from 0-24 hours, 0-360 degrees, or 0-2π radians.
    """
    ra_decimal_hours = hours + minutes/60 + seconds/3600 
    degrees = ra_decimal_hours * 15
    radians = degrees * (DPI/180)    

    return (ra_decimal_hours, degrees, radians)


In [38]:
# Format 1: HH:MM (Noon on 2000-01-01)
date_str_1 = "2025-09-26 21:00:00"
Y1, M1, D1 = parse_date_to_YMD_flexible(date_str_1)

jd1 = calculate_julian_day(Y1, M1, D1)
print(f"1 Jan 2000 12:00:00 -> JD: {jd1}")


theta_degrees = calculate_mean_sidereal_time(jd1)

# At the epoch J2000.0 (JD 2451545.0), T=0, so the result should be 280.46061837 degrees
print(f"JD: {jd1}")
print(f"Mean Sidereal Time (Degrees): {theta_degrees:.8f}°")

1 Jan 2000 12:00:00 -> JD: 2460945.375
JD: 2460945.375
Mean Sidereal Time (Degrees): 320.91550495°


In [39]:
Hours, Minutes, Seconds = degrees_to_hms(theta_degrees)
print(f"Decimal Degrees: {theta_degrees}")
print(f"Sidereal Time (H:M:S): {Hours}h {Minutes}m {Seconds:.4f}s")
# Expected Output: 11h 39m 5.0673s (or similar depending on precision)

Decimal Degrees: 320.915504953824
Sidereal Time (H:M:S): 21h 23m 39.7212s


In [17]:
longitude = 321.3126 
if longitude <= 180:
    longitude = longitude
elif longitude > 180:
    longitude = longitude - 360
   

In [40]:
# Y28,321.3126,-8.788707,414.679,Brazil,"OASI, Nova Itacuruba"
GMST = theta_degrees
longitude_360 = 321.3126 

# Convert longitude to astronomical convention (-180 to +180)
if longitude_360 <= 180:
    longitude = longitude_360
else:
    longitude = longitude_360 - 360

# Calculate LST
# Make sure GMST is in HOURS, not degrees
LST = (GMST/15 + (longitude/15)) % 24  # longitude/15 converts degrees to hours

print(f"LST: {LST:.6f} hours")

LST: 18.815207 hours


In [58]:
ra_string = "19 36 00.59"
ra_hours, ra_minutes, ra_seconds = parse_ra_string(ra_string)

ra_decimal_hours, ra_decimal_degrees, ra_radians = convert_ra_components(ra_hours, ra_minutes, ra_seconds)


H = LST - ra_decimal_hours

H

-0.784956891967294

In [42]:
def parse_dec_string(dec_string):
    """
    Parse declination string into component values.
   
    Takes a declination string in format "±DD MM SS.S" and extracts
    the degrees, minutes, and seconds as separate float values.
   
    Args:
        dec_string (str): Declination in format "±DD MM SS.S"
                         (e.g., "-16 42 58.0" or "+38 20 56.8")
   
    Returns:
        tuple: A tuple of four values (sign, degrees, minutes, seconds)
               - sign (int): +1 for positive, -1 for negative
               - degrees (float): Absolute degrees value
               - minutes (float): Arcminutes value  
               - seconds (float): Arcseconds value
               e.g., (-1, 16.0, 42.0, 58.0)
   
    Example:
        >>> parse_dec_string("-16 42 58.0")
        (-1, 16.0, 42.0, 58.0)
        >>> parse_dec_string("+38 20 56.8")
        (1, 38.0, 20.0, 56.8)
    """
    try:
        dec_string = dec_string.strip()
        
        # Determine sign
        if dec_string.startswith('-'):
            sign = -1
            dec_string = dec_string[1:]  # Remove the minus sign
        elif dec_string.startswith('+'):
            sign = 1
            dec_string = dec_string[1:]  # Remove the plus sign
        else:
            sign = 1  # Default to positive if no sign given
        
        parts = dec_string.split()
        if len(parts) != 3:
            raise ValueError("DEC string must be in the format '±DD MM SS.S'")
       
        dec_degrees = float(parts[0])
        dec_minutes = float(parts[1])
        dec_seconds = float(parts[2])
       
        return (sign, dec_degrees, dec_minutes, dec_seconds)
   
    except ValueError as e:
        print(f"Error parsing DEC string: {e}")
        return None


def convert_dec_components(sign, degrees, minutes, seconds):
    """
    Convert declination components to decimal degrees and radians.
   
    Takes individual sign, degrees, minutes, and seconds components of declination
    and converts them to decimal degrees and radians.
   
    Args:
        sign (int): Sign of declination (+1 or -1)
        degrees (float): Degrees component of declination
        minutes (float): Arcminutes component of declination  
        seconds (float): Arcseconds component of declination
   
    Returns:
        tuple: A tuple of two floats:
               - decimal_degrees (float): DEC in decimal degrees
               - radians (float): DEC in radians
   
    Example:
        >>> convert_dec_components(-1, 16.0, 42.0, 58.0)
        (-16.71611111111111, -0.29177...)
   
    Note:
        Declination ranges from -90° to +90° or -π/2 to +π/2 radians.
        Negative values indicate southern declinations.
    """
    # Convert to decimal degrees (note: all components use the same sign)
    dec_decimal_degrees = sign * (degrees + minutes/60 + seconds/3600)
    
    # Convert to radians (assuming DPI is defined as math.pi elsewhere)
    import math
    radians = dec_decimal_degrees * (math.pi/180)
    
    return (dec_decimal_degrees, radians)


# Example usage and testing
if __name__ == "__main__":
    # Test cases
    test_cases = [
        "-16 42 58.0",  # Sirius
        "+38 20 56.8",  # Vega  
        "-26 19 27.2",  # Alpha Centauri
        "90 00 00.0",   # North Celestial Pole
        "-90 00 00.0"   # South Celestial Pole
    ]
    
    for dec_str in test_cases:
        print(f"\nTesting: {dec_str}")
        parsed = parse_dec_string(dec_str)
        if parsed:
            sign, deg, min_val, sec = parsed
            print(f"  Parsed: sign={sign}, deg={deg}, min={min_val}, sec={sec}")
            
            decimal_deg, radians = convert_dec_components(sign, deg, min_val, sec)
            print(f"  Decimal degrees: {decimal_deg:.8f}°")
            print(f"  Radians: {radians:.8f}")


Testing: -16 42 58.0
  Parsed: sign=-1, deg=16.0, min=42.0, sec=58.0
  Decimal degrees: -16.71611111°
  Radians: -0.29175118

Testing: +38 20 56.8
  Parsed: sign=1, deg=38.0, min=20.0, sec=56.8
  Decimal degrees: 38.34911111°
  Radians: 0.66931825

Testing: -26 19 27.2
  Parsed: sign=-1, deg=26.0, min=19.0, sec=27.2
  Decimal degrees: -26.32422222°
  Radians: -0.45944435

Testing: 90 00 00.0
  Parsed: sign=1, deg=90.0, min=0.0, sec=0.0
  Decimal degrees: 90.00000000°
  Radians: 1.57079633

Testing: -90 00 00.0
  Parsed: sign=-1, deg=90.0, min=0.0, sec=0.0
  Decimal degrees: -90.00000000°
  Radians: -1.57079633


In [57]:
sign, dec_degrees, dec_minutes, dec_seconds = parse_dec_string('-17 31 13.8')

dec_decimal_degrees, radians = convert_dec_components(sign, dec_degrees, dec_minutes, dec_seconds)

dec_decimal_degrees, radians

(-17.5205, -0.3057904115956665)

In [59]:
#sin(altitude) = sin(φ)sin(δ) + cos(φ)cos(δ)cos(H).


import math

# Observatory coordinates
latitude = -8.788707  # Negative because it's south
phi = math.radians(latitude)  # Convert to radians for calculation

# Example star (let's say declination = +20°)
delta = math.radians(dec_decimal_degrees)  # Star's declination

# Example hour angle (let's say H = 30°)
H = math.radians(H*15)

# Calculate altitude
sin_altitude = (math.sin(phi) * math.sin(delta) + 
                math.cos(phi) * math.cos(delta) * math.cos(H))

altitude_radians = math.asin(sin_altitude)
altitude_degrees = math.degrees(altitude_radians)

print(f"Altitude: {altitude_degrees:.6f}°")

Altitude: 75.599349°


In [47]:
# Apply elevation corrections
observatory_elevation = 414.679  # meters (from your data)

# 1. Horizon depression correction
horizon_depression = math.sqrt(observatory_elevation / 6371000)  # radians
altitude_corrected = altitude_radians + horizon_depression
math.degrees(altitude_corrected)

76.31547914442514